# ETL, Matching, KPI construction and Dashboard-ready exports (English)

This notebook implements a reproducible pipeline to:
- Load the two provided sheets (RFI / DCM and GA).
- Clean and harmonize fields (dates, numeric types, missing values).
- Extract features from GA Ad Content and RFI Creative/Placement.
- Heuristically match GA rows to RFI rows with confidence score.
- Build monthly KPIs and a consolidated CSV ready for BI tools.
- Export ambiguous matches for manual validation.

Follow the cells in order. Adjust file paths if the Excel file is not in the notebook folder.

In [ ]:
# Cell 1: Imports and settings
import os
from pathlib import Path
import re
import json
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', context='notebook')
pd.set_option('display.max_columns', 200)


In [ ]:
# Cell 2: Robust file detection and load
EXCEL_NAME = 'Analytics Test & Data Set - Instructions.xlsx'
wb_path = Path(EXCEL_NAME)
if not wb_path.exists():
    raise FileNotFoundError(f"Expected file not found in working dir: {EXCEL_NAME}")

# RFI (DCM) sheet assumed named 'RFI Data'
rfi_df = pd.read_excel(wb_path, sheet_name='RFI Data')
# GA sheet may have header rows to skip; original notebook used skiprows=3
ga_df = pd.read_excel(wb_path, sheet_name='Raw GA Data', skiprows=3)

print('Loaded', EXCEL_NAME)
print('RFI shape:', rfi_df.shape)
print('GA shape:', ga_df.shape)


In [ ]:
# Cell 3: Basic cleaning and normalization
def safe_to_period(df, col, fmt=None):
    if col not in df.columns:
        return df
    if fmt:
        df[col] = pd.to_datetime(df[col].astype(str), format=fmt, errors='coerce').dt.to_period('M')
    else:
        df[col] = pd.to_datetime(df[col], errors='coerce').dt.to_period('M')
    return df

# Normalize month columns
rfi_df = safe_to_period(rfi_df, 'Month', fmt='%Y-%m')
ga_df = safe_to_period(ga_df, 'Month of Year', fmt='%Y%m')

# Fill categorical unknowns to avoid grouping errors
for c in ['Platform Type', 'Device Category']:
    if c in rfi_df.columns:
        rfi_df[c] = rfi_df[c].fillna('Unknown')
    if c in ga_df.columns:
        ga_df[c] = ga_df[c].fillna('Unknown')

# Ensure numeric columns exist and are numeric
for c in ['Impressions', 'Clicks']:
    if c in rfi_df.columns:
        rfi_df[c] = pd.to_numeric(rfi_df[c], errors='coerce').fillna(0)
for c in ['Sessions', 'Users', 'Pageviews']:
    if c in ga_df.columns:
        ga_df[c] = pd.to_numeric(ga_df[c], errors='coerce').fillna(0)

rfi_df.head(2)

In [ ]:
# Cell 4: Extractors and helpers
def safe_str(x):
    return '' if pd.isna(x) else str(x)

def strip_digits(s):
    if pd.isna(s) or s == '':
        return None
    m = re.search(r'(\d{3,4})', str(s))
    return m.group(1) if m else None

def extract_width_from_dims(d):
    if pd.isna(d) or d == '':
        return None
    m = re.search(r'(\d{2,4})\s*[xX]\s*(\d{2,4})', str(d))
    return int(m.group(1)) if m else None

def extract_ga_parts(ad_content):
    s = safe_str(ad_content)
    parts = s.split('.')
    width = None
    if parts and re.fullmatch(r'\d{2,4}', parts[-1]):
        try:
            width = int(parts[-1])
        except Exception:
            width = None
    nums = re.findall(r'\d{3,4}', s)
    num = nums[0] if nums else None
    token = parts[0].lower() if parts else s.lower()
    return {'ga_token': token, 'ga_num': num, 'ga_width': width, 'ga_raw': s}

def extract_rfi_parts(row):
    placement = safe_str(row.get('Placement - DCM', ''))
    creative = safe_str(row.get('Creative', ''))
    dims = safe_str(row.get('Creative Dimensions', ''))
    nums = re.findall(r'\d{3,4}', placement + ' ' + creative)
    num = nums[0] if nums else None
    width = extract_width_from_dims(dims)
    return {'rfi_placement': placement.lower(), 'rfi_creative': creative.lower(), 'rfi_num': num, 'rfi_width': width, 'rfi_dims': dims}

def similarity(a, b):
    return SequenceMatcher(None, safe_str(a), safe_str(b)).ratio()


In [ ]:
# Cell 5: Parse GA and RFI, create working tables
ga_parsed = ga_df.copy().reset_index().rename(columns={'index':'ga_index'})
rfi_parsed = rfi_df.copy().reset_index().rename(columns={'index':'rfi_index'})

ga_parsed['month'] = ga_parsed.get('Month of Year', ga_parsed.get('month')).astype(str)
rfi_parsed['month'] = rfi_parsed.get('Month', rfi_parsed.get('month')).astype(str)

ga_parts = ga_parsed.get('Ad Content', pd.Series(['']*len(ga_parsed))).apply(extract_ga_parts).apply(pd.Series)
ga_parsed = pd.concat([ga_parsed, ga_parts], axis=1)

rfi_parts = rfi_parsed.apply(extract_rfi_parts, axis=1).apply(pd.Series)
rfi_parsed = pd.concat([rfi_parsed, rfi_parts], axis=1)

ga_parsed['ga_num_stripped'] = ga_parsed['ga_num'].apply(strip_digits)
ga_parsed['ga_width'] = ga_parsed['ga_width'].apply(lambda x: int(x) if pd.notna(x) and str(x).isdigit() else None)
rfi_parsed['rfi_num'] = rfi_parsed['rfi_num'].apply(strip_digits)
rfi_parsed['rfi_width'] = rfi_parsed['rfi_width'].fillna(rfi_parsed['rfi_dims'].apply(extract_width_from_dims))

print('Parsed GA rows:', len(ga_parsed), 'Parsed RFI rows:', len(rfi_parsed))


In [ ]:
# Cell 6: Refactored matching function
def match_ga_to_rfi(ga_df, rfi_df, month_strict=True):
    rfi_index = rfi_df.copy()
    matches = []
    for _, ga in ga_df.iterrows():
        candidates = rfi_index
        # restrict by month if requested and month column exists
        if month_strict and pd.notna(ga.get('month')):
            cand_month = candidates[candidates['month'] == ga['month']]
            if not cand_month.empty:
                candidates = cand_month

        best = None; best_score = 0.0
        ga_num = safe_str(ga.get('ga_num_stripped') or '')

        # 1) numeric exact (prefer width match)
        if ga_num:
            cand_num = candidates[candidates['rfi_num'] == ga_num]
            if not cand_num.empty:
                if pd.notna(ga.get('ga_width')):
                    cand_num_w = cand_num[cand_num['rfi_width'] == ga['ga_width']]
                    if not cand_num_w.empty:
                        best = cand_num_w.iloc[0]; best_score = 1.0
                    else:
                        best = cand_num.iloc[0]; best_score = 0.9
                else:
                    best = cand_num.iloc[0]; best_score = 0.9

        # 2) prefix/token match
        if best_score < 0.8:
            token = safe_str(ga.get('ga_token','')).lower()
            src_pref = re.match(r'^([a-z]{2,4})', token)
            if src_pref:
                pref = src_pref.group(1)
                cand_pref = candidates[candidates['rfi_placement'].str.contains(pref, na=False) | candidates['rfi_creative'].str.contains(pref, na=False)]
                if not cand_pref.empty:
                    if pd.notna(ga.get('ga_width')):
                        cand_pref_w = cand_pref[cand_pref['rfi_width'] == ga['ga_width']]
                        if not cand_pref_w.empty:
                            best = cand_pref_w.iloc[0]; best_score = 0.85
                        else:
                            best = cand_pref.iloc[0]; best_score = 0.75
                    else:
                        best = cand_pref.iloc[0]; best_score = 0.75

        # 3) width-only
        if best_score < 0.6 and pd.notna(ga.get('ga_width')):
            cand_w = candidates[candidates['rfi_width'] == ga['ga_width']]
            if not cand_w.empty:
                best = cand_w.iloc[0]; best_score = 0.65

        # 4) fuzzy fallback (allow broader search)
        if best_score < 0.5:
            candidates_all = rfi_index.copy()
            if ga_num:
                cand_num = candidates_all[candidates_all['rfi_num'] == ga_num]
                if not cand_num.empty:
                    if pd.notna(ga.get('ga_width')):
                        cand_num_w = cand_num[cand_num['rfi_width'] == ga['ga_width']]
                        best = (cand_num_w.iloc[0] if not cand_num_w.empty else cand_num.iloc[0]); best_score = 0.88
                    else:
                        best = cand_num.iloc[0]; best_score = 0.88
            if best is None:
                candidates_all['sim_creative'] = candidates_all['rfi_creative'].apply(lambda x: similarity(ga.get('Ad Content',''), x))
                candidates_all['sim_placement'] = candidates_all['rfi_placement'].apply(lambda x: similarity(ga.get('Ad Content',''), x))
                candidates_all['sim_max'] = candidates_all[['sim_creative','sim_placement']].max(axis=1)
                top = candidates_all.sort_values('sim_max', ascending=False).head(1)
                if not top.empty and top['sim_max'].iloc[0] > 0.55:
                    best = top.iloc[0]; best_score = float(top['sim_max'].iloc[0]) * 0.5

        if best is not None:
            matches.append({
                'ga_index': int(ga['ga_index']),
                'Ad Content': ga.get('Ad Content'),
                'ga_token': ga.get('ga_token'),
                'ga_num': ga.get('ga_num_stripped'),
                'ga_width': ga.get('ga_width'),
                'month': ga.get('month'),
                'rfi_index': int(best['rfi_index']) if pd.notna(best.get('rfi_index')) else None,
                'rfi_placement': best.get('rfi_placement'),
                'rfi_creative': best.get('rfi_creative'),
                'rfi_num': best.get('rfi_num'),
                'rfi_width': best.get('rfi_width'),
                'confidence': round(best_score, 3)
            })
        else:
            matches.append({
                'ga_index': int(ga['ga_index']),
                'Ad Content': ga.get('Ad Content'),
                'ga_token': ga.get('ga_token'),
                'ga_num': ga.get('ga_num_stripped'),
                'ga_width': ga.get('ga_width'),
                'month': ga.get('month'),
                'rfi_index': None,
                'rfi_placement': None,
                'rfi_creative': None,
                'rfi_num': None,
                'rfi_width': None,
                'confidence': 0.0
            })
    return pd.DataFrame(matches)


In [ ]:
# Cell 7: Run matching and merge back
matches_df = match_ga_to_rfi(ga_parsed, rfi_parsed, month_strict=True)
rfi_index_full = rfi_parsed.copy()
ga_with_match = ga_parsed.merge(matches_df[['ga_index','rfi_index','confidence']], on='ga_index', how='left') \
                  .merge(rfi_index_full, left_on='rfi_index', right_on='rfi_index', how='left', suffixes=('_ga','_rfi'))

print('Matches produced:', len(matches_df))
print('High-confidence matches:', matches_df['confidence'].ge(0.75).sum())
matches_df.head(5)


In [ ]:
# Cell 8: KPI construction and BI-ready consolidation
# Monthly aggregates
rfi_monthly = rfi_df.groupby('Month')[['Impressions','Clicks']].sum().reset_index()
rfi_monthly['CTR'] = rfi_monthly.apply(lambda r: (r['Clicks'] / r['Impressions']) if r['Impressions']>0 else np.nan, axis=1)

ga_monthly = ga_df.groupby('Month of Year')[['Sessions','Users']].sum().reset_index()
ga_monthly['Engagement Rate'] = ga_monthly.apply(lambda r: (r['Sessions'] / r['Users']) if r['Users']>0 else np.nan, axis=1)

# Normalize month columns to same string key
rfi_monthly['month_key'] = rfi_monthly['Month'].astype(str)
ga_monthly['month_key'] = ga_monthly['Month of Year'].astype(str)

dashboard_df = pd.merge(rfi_monthly, ga_monthly, left_on='month_key', right_on='month_key', how='inner', suffixes=('_rfi','_ga'))
dashboard_df['Sessions_per_Click'] = dashboard_df.apply(lambda r: (r['Sessions']/r['Clicks']) if r['Clicks']>0 else np.nan, axis=1)
dashboard_df['Efficiency_Index'] = dashboard_df['CTR'] * dashboard_df['Engagement Rate']

dashboard_csv = Path('consolidated_for_dashboard.csv')
dashboard_df.to_csv(dashboard_csv, index=False)
matches_df.to_csv('site_creative_match_suggestions_v2.csv', index=False)
ga_with_match.to_csv('consolidated_ga_rfi_join_v2.csv', index=False)

print('Saved consolidated_for_dashboard.csv (rows =', len(dashboard_df),')')


In [ ]:
# Cell 9: EDA quick charts (guarded) and ambiguous export
import matplotlib.dates as mdates

if 'confidence' in matches_df.columns and not matches_df.empty:
    display(matches_df['confidence'].describe())
    ambig = matches_df[matches_df['confidence'].between(0.2,0.75)].copy()
    if not ambig.empty:
        ambig.to_csv('ambiguous_match_suggestions.csv', index=False)
        print('Saved ambiguous_match_suggestions.csv (for manual review)')

# Simple KPI printout
print('Total Impressions:', int(rfi_df['Impressions'].sum()))
print('Total Clicks:', int(rfi_df['Clicks'].sum()))
print('Total GA Sessions:', int(ga_df['Sessions'].sum()))
print('Avg CTR (%):', (rfi_monthly['CTR'].mean()*100).round(2))


## Documentation (EN)

1) Approach
- Load and harmonize dates and key fields.
- Extract tokens, numeric ids and widths from GA Ad Content and RFI Creative/Placement.
- Match heuristically using a prioritized set of rules (numeric exact, prefix, width, fuzzy) and assign confidence.
- Build month-level KPIs and export a consolidated CSV for BI tools (PowerBI, Tableau, Data Studio).

2) Deliverables produced by this notebook
- consolidated_for_dashboard.csv — monthly KPIs and derived metrics ready for dashboarding.
- site_creative_match_suggestions_v2.csv — suggested joins with confidence scores.
- ambiguous_match_suggestions.csv — medium-confidence rows for manual review (if any).
- ETL notebook (this file) documenting the steps and code.

3) Dashboard proposal
- Filters: date range (month), site, creative, device, confidence bucket.
- KPI cards: Impressions, Clicks, CTR (weighted), Sessions, Users, Engagement Rate, Sessions/Click, Efficiency Index.
- Visuals: trend lines (Impressions, Clicks, CTR, Sessions), CTR vs Engagement scatter + regression, top-N tables, width heatmap, ambiguous table.

4) Additional data requested for deeper insights
- Cost (media spend) by creative/placement to compute CPC, CPA, ROAS.
- Landing page URLs and full UTM params for better attribution.
- Timestamps or click-level logs for lag analysis (click → session).
- Conversion events / goals to measure outcome vs traffic.

5) Time estimate to deliver final dashboard
- Cleaned exports + notebook: 3-5 hours.
- Interactive PowerBI dashboard (basic filters + visuals): 4-8 hours.
- Polished report and validation: 2-4 hours.
